FC Transport

**Decision Variables:** Let $X_1, \cdots, X_7$ denote whether to use each FC. (Binary)

**Objective and constraints:**

$$\begin{aligned}
\text{Minimize} && X_1+X_2+\cdots+X_7 \\
\text{s.t.} && X_2+X_5+X_6+X_7 & \ge 1\\
&& X_3+X_4 & \ge 1\\
&& X_3 & \ge 1 \\
&& X_1+X_2+X_4+X_6 & \ge 1 \\
&& X_5 + X_7 & \ge 1\\
&& X_4 &\le X_1 \\
&& X_2+X_3 & \le 1
\end{aligned}$$

**a)** Implement the above using Gurobi

In [1]:
atLeastOne=[[2,5,6,7],[3,4],[3],[1,2,4,6],[5,7]]
from gurobipy import Model, GRB
mod=Model()
x=mod.addVars(range(1,8),vtype=GRB.BINARY,name='X')
mod.setObjective(sum(x[i] for i in range(1,8)))
for listOfFCs in atLeastOne:
    mod.addConstr(sum(x[i] for i in listOfFCs)>=1)
mod.addConstr(x[4]<=x[1])
mod.addConstr(x[2]+x[3]<=1)
mod.update()
mod.write('PS8-Q1.lp')
%cat 'PS8-Q1.lp'
#!type PS8-Q1.lp   (on Windows)

Using license file /home/pengshi/gurobi.lic
Academic license - for non-commercial use only
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  X[1] + X[2] + X[3] + X[4] + X[5] + X[6] + X[7]
Subject To
 R0: X[2] + X[5] + X[6] + X[7] >= 1
 R1: X[3] + X[4] >= 1
 R2: X[3] >= 1
 R3: X[1] + X[2] + X[4] + X[6] >= 1
 R4: X[5] + X[7] >= 1
 R5: - X[1] + X[4] <= 0
 R6: X[2] + X[3] <= 1
Bounds
Binaries
 X[1] X[2] X[3] X[4] X[5] X[6] X[7]
End


**b)** Solve the MIP and print the minimum number of FCs needed, as well as where to stock the items. 

In [2]:
mod.setParam('OutputFlag',False)
mod.optimize()
print('Minimum # of FCs needed:',int(mod.objval))
print('Stock item in the following:')
for i in range(1,8):
    if x[i].x==1:
        print(f'\tFC{i}')

Minimum # of FCs needed: 3
Stock item in the following:
	FC1
	FC3
	FC7


Optimal earnings

Nature's Best Frozen Foods company produces four different mixes of frozen ready-to-eat vegetables. The mixes consist of five different vegetables: carrots, mushrooms, green peppers, broccoli, and corn. The company manufacturers four different mixes, each sold in 10 oz. bags. The mixes are "Stir Fry", "Barbecue", "Hearty Mushrooms" and "Veggie Crunch," and their contributions to earnings (per bag) are 0.22, 0.20, 0.18 and 0.18 respectively. The monthly supplies of carrots, mushrooms, green peppers, broccoli and corn are 150,000 oz., 80,000 oz., 135,000 oz., 140,000 oz., and 150,000 oz. per month, respectively. The compositions of the mixes are shown in the table below. For example, one bag of "Stir Fry" mix contains 2.5 oz. of carrots, 3.0 oz. of mushrooms, 2.5 oz. of green peppers, 2.0 oz. of broccoli, and no corn. 

|` `| Stir Fry | Barbecue | Hearty Mushrooms | Veggie Crunch |
|--|--|--|--|--|
|Carrots | 2.5 | 2.0 | - | 2.5 |
|Mushrooms | 3.0 | - | 4.0 | - |
|Green Peppers | 2.5 | 2.0 | 3 | 2.5 |
|Broccoli | 2.0 | 3.0 | 3.0 | 2.5 |
|Corn | - | 3.0 | - | 2.5 |

**a)** Formulate a LP to determine the optimal monthly production of each mix to maximize total earnings. 

### Step 1. English description

**Decision:** How much of each mix to produce in a month.

**Objective:** Maximize total earnings.

**Constraints:** For each of the five ingredients, the total amount used must be less than equal to the supply available.

### Step 2. Concrete formulation

**Decision variables:** 

- $S$: amount of Stir Fry mixes to produce (Continuous); 
- $B$: amount of Barbeque (Continuous); 
- $H$: amount  of Hearty Mushrooms (Continuous);
- $V$: amount  of Veggie Crunch (Continuous).

**Objective and constraints:**

$$\begin{aligned}
\text{Maximize:} && 0.22S+0.2B+0.18H+0.18V \\
\text{Subject to:} \\
\text{(Supply of Carrots)} && 2.5S+2.0B+2.5V & \le 150000 \\
\text{(Supply of Mushrooms)} && 3S + 4H & \le 80000 \\
\text{(Supply of Green peppers)} && 2.5S+2B+3H+2.5V & \le 135000 \\
\text{(Supply of Broccoli)} && 2S+3B+3H+2.5V & \le 140000 \\
\text{(Supply of Corn)} && 3B +2.5V & \le 150000 \\
\text{} && S,B,H,V & \ge 0
\end{aligned}$$

**b)** Write Gurobi code to implement the above LP. 

In [3]:
products=['Stir Fry','Barbeque','Hearty Mushrooms','Veggie Crunch']
import pandas as pd
earnings=pd.Series([.22,.2,.18,.18],index=products)
earnings

Stir Fry            0.22
Barbeque            0.20
Hearty Mushrooms    0.18
Veggie Crunch       0.18
dtype: float64

In [4]:
ingredients=['Carrots','Mushrooms','Green peppers','Broccoli','Corn']
supply=pd.Series([150000,80000,135000,140000,150000],index=ingredients)
supply

Carrots          150000
Mushrooms         80000
Green peppers    135000
Broccoli         140000
Corn             150000
dtype: int64

In [5]:
usage=pd.DataFrame([[2.5,2,0,2.5],\
                    [3,0,4,0],\
                    [2.5,2,3,2.5],\
                    [2,3,3,2.5],\
                    [0,3,0,2.5]],\
                  index=ingredients, columns=products)
usage

,Stir Fry,Barbeque,Hearty Mushrooms,Veggie Crunch
Carrots,2.5,2,0,2.5
Mushrooms,3.0,0,4,0.0
Green peppers,2.5,2,3,2.5
Broccoli,2.0,3,3,2.5
Corn,0.0,3,0,2.5


In [6]:
from gurobipy import Model,GRB
mod=Model()
x=mod.addVars(products)
mod.setObjective(sum(earnings.loc[p]*x[p] for p in products),sense=GRB.MAXIMIZE)
for i in ingredients:
    mod.addConstr(sum(usage.loc[i,p]*x[p] for p in products)<=supply.loc[i])
mod.setParam('OutputFlag',False) 
mod.optimize()
print('Maximum earning:',round(mod.objval,2))
print('Production plan:')
for p in products:
    print(f'\t{p}: {round(x[p].x,2)}')

Maximum earning: 11813.33
Production plan:
	Stir Fry: 26666.67
	Barbeque: 18333.33
	Hearty Mushrooms: 0.0
	Veggie Crunch: 12666.67


Optimal Advertising Plan

SALS Marketing Inc. is developing an advertising campaign for a large consumer goods corporation. An advertising plan specifies how many units of each kind of advertisement to purchase. SALS has promised a plan that will yield the highest possible “exposure rating,” which is a measure of the ability to reach the appropriate demographic group and generate demand. The options for advertisements with their respective costs (per unit of advertising) and per-unit exposure ratings are given in the table below (K stands for thousands).

| Category | Subcategory | Cost/Unit | Exposure/Unit |
|--|--|--|--|
| Magazines | Literary | \$7.5 K | 15 K |
| ` ` | News | \$10 K | 22.5 K |
| ` ` | Topical | \$15 K | 24 K |
| Newspapers |  Morning | \$2 K | 37.5 K |
|` `  | Evening | \$3 K | 75 K |
| Television | Morning | \$20 K | 275 K |
| ` ` | Midday | \$10 K | 180 K |
| ` `  | Evening | \$60 K | 810 K |
| Radio | Morning | \$15 K | 180 K |
| ` ` | Midday | \$15 K | 17 K |
| ` ` | Evening | \$10 K | 16 K |

Of course, certain restrictions exist for the advertising campaign. The client corporation has budgeted 800,000 dollars for the campaign, but to restrict overexposure to any particular audience it wants no more than 300,000 dollars put into any one category (Magazine, Newspaper, etc.). Also, to ensure a broad range of exposure, at least 100,000 dollars must be spent in each category. Finally, one has to purchase an integer number of units of each kind of advertisement, as no fractional units are allowed.

**a)** Formulate a linear optimization problem to determine the optimal advertising plan.

### Step 1. English Description

**Decision:** How many units of each kind of advertisement to purchase.

**Objective:** Maximize the total exposure rating. 

**Constraints:** 

- Not spend more than 800 K in total.
- Spend between 100 K and 300 K in each category.

### Step 2. Concrete Formulation

**Decision variables:**

- $M_L$, $M_N$, $M_T$: how many units of literary, news, and topical magazines to include. (Integer)
- $N_M$, $N_E$: analogous decision variables for newspapers. (Integer)
- $T_M$, $T_D$, $T_E$: analogous decision variables for television. (Integer)
- $R_M$, $R_D$, $R_E$: analogous decision variables for radio. (Integer)

For convenience, we measure money in thousands of dollars.

**Objective and constraints:**

$$\begin{aligned}
\text{Maximize:} && 15M_L+22.5M_N+24M_T+37.5N_M+75N_E+  \\
&& 275T_M+180T_D+810T_E+180R_M+17R_D+16R_E \\
\text{Subject to:} \\
\text{(Spending in M)} && 100\le 7.5M_L+10M_N+15M_T &\le 300 \\
\text{(Spending in N)} && 100 \le 2N_M + 3N_E &\le 300  \\
\text{(Spending in T)} && 100 \le 20T_M+10T_D+60T_E &\le 300 \\
\text{(Spending in R)} && 100 \le 15R_M+15R_D+10R_E &\le 300 \\
\text{(Total spending)} && 7.5M_L+10M_N+15M_T+3N_E+2N_M+20T_M+ \\ 
&& 10T_D+60T_E+15R_M+15R_D+10R_E &\le 800\\
\text{(Non-negativity)} && M_L,M_N,M_T,\cdots,R_E & \ge 0
\end{aligned}$$

**b)** Write Gurobi code to implement the above formulation. Your code should read in the data from the following data structures rather than hard code in the numbers. For convenience, all numerical values are in the units of K (thousands). 

The outputs should be in the same format as the sample outputs below. Note: Gurobi might output strangely formatted numbers like `-0`, and you can make it `0` by converting it to `int`.

In [7]:
# Constructing the subcategories
subcat={}
subcat['Magazines']=['Literary Mag.','News Mag.','Topical Mag.']
subcat['Newspapers']=['Morning News','Evening News']
subcat['Television']=['Morning TV','Midday TV','Evening TV']
subcat['Radio']=['Morning Radio','Midday Radio','Evening Radio']
subcat

{'Magazines': ['Literary Mag.', 'News Mag.', 'Topical Mag.'],
 'Newspapers': ['Morning News', 'Evening News'],
 'Television': ['Morning TV', 'Midday TV', 'Evening TV'],
 'Radio': ['Morning Radio', 'Midday Radio', 'Evening Radio']}

In [8]:
import pandas as pd
allSubCat=subcat['Magazines']+subcat['Newspapers']+subcat['Television']+subcat['Radio']
data=pd.DataFrame([[7.5,15],[10,22.5],[15,24],\
                 [2,37.5],[3,75],
                 [20,275],[10,180],[60,810],\
                 [15,180],[15,17],[10,16]],\
                 index=allSubCat,columns=['Cost','Exposure'])
data

,Cost,Exposure
Literary Mag.,7.5,15.0
News Mag.,10.0,22.5
Topical Mag.,15.0,24.0
Morning News,2.0,37.5
Evening News,3.0,75.0
Morning TV,20.0,275.0
Midday TV,10.0,180.0
Evening TV,60.0,810.0
Morning Radio,15.0,180.0
Midday Radio,15.0,17.0


In [9]:
# Sample Solution
from gurobipy import Model, GRB
cost=data['Cost']
exposure=data['Exposure']
mod=Model()
x=mod.addVars(allSubCat,vtype=GRB.INTEGER)
mod.setObjective(sum(exposure.loc[s]*x[s] for s in allSubCat),sense=GRB.MAXIMIZE)
for category in subcat:
    spending=sum(cost.loc[s]*x[s] for s in subcat[category])
    mod.addConstr(spending>=100)
    mod.addConstr(spending<=300)
mod.addConstr(sum(cost.loc[s]*x[s] for s in allSubCat)<=800)
mod.setParam('outputflag',False)
mod.optimize()
print(f'Maximum total exposure (in thousands):',mod.objval)
print('# of units to purchase:')
for s in allSubCat:
    print(f'\t{s}: {int(x[s].x)}')

Maximum total exposure (in thousands): 14235.0
# of units to purchase:
	Literary Mag.: 0
	News Mag.: 10
	Topical Mag.: 0
	Morning News: 0
	Evening News: 98
	Morning TV: 0
	Midday TV: 30
	Evening TV: 0
	Morning Radio: 7
	Midday Radio: 0
	Evening Radio: 0
